In [ ]:
%run GraphFamilies.ipynb

In [10]:
# Get list of graphs from graph families

#SETTINGS:

# Maximum size hole to include
maxhole = 4

# Maximum length path to include
maxpath = 2

# TRUE: Use connected graph families
# FALSE: Use disconnected graph familise
connected = True 

#############
graph_family_list = AllFamilies(maxhole, maxpath, connected)
print(f"{len(graph_family_list)} graphs included for testing")

27 graphs included for testing


In [15]:
# Build full table of all induced subgraphs in list of graphs in the following form:
# Column 1: Graph
# Column 2: Full list of induced subgraphs of this graph
# Table is stored as a list of tuples

def build_graph_table():
    graph_table = []

    # Iterate each graph
    for G in graph_family_list:
        G_sg_list = []

        # Iterate each induced subgraph of G
        for vertex_subset in Subsets(G.vertices()):
            current_sg = G.subgraph(vertex_subset)

            add_flag = True
            for H in G_sg_list:
                if (current_sg.order() != H.order()) or (current_sg.size() != H.size()): #Different number of vertices/edges, no need to test isomorphism
                    continue
                elif H.is_isomorphic(current_sg):
                    add_flag = False
                    break 
            
            if add_flag == True:
                G_sg_list.append(current_sg)
        
        graph_table.append((G, G_sg_list))
    
    return graph_table

graph_table = build_graph_table()


[(Graph on 7 vertices, [Subgraph of (): Graph on 0 vertices, Subgraph of (): Graph on 1 vertex, Subgraph of (): Graph on 2 vertices, Subgraph of (): Graph on 2 vertices, Subgraph of (): Graph on 3 vertices, Subgraph of (): Graph on 3 vertices, Subgraph of (): Graph on 3 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 4 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 5 vertices, Subgraph of (): Graph on 6 vertices, Subgraph of (): Graph on 6 vertices, Subgraph of (): Graph on 6 vertices, Subgraph of (): Graph on 7 vertices]), (Graph on 6 vertices, [Subgraph of (): Graph on 0 v